## Importing required Libraries

In [50]:
# Importing libraries required for handling the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing metrics library
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Importing libraries for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Importing library for splitting the data
from sklearn.model_selection import train_test_split

# Importing regression models from scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

## Reading the data

In [52]:
df = pd.read_csv("https://raw.githubusercontent.com/Brianc482/731_Group_Project/main/Data/Fifa18.csv")
df.head(1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,€95.5M,€565K,2228,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88,89.0,53.0,62.0,91.0,82.0,20801,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0


## Dropping columns

In [53]:
df.drop(["Unnamed: 0","Flag","Club","Photo","Club Logo","Value","Wage"],axis=1,inplace=True)
df.head()

,Name,Age,Nationality,Overall,Potential,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,Cristiano Ronaldo,32,Portugal,94,94,2228,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88,89.0,53.0,62.0,91.0,82.0,20801,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,L. Messi,30,Argentina,93,93,2154,92,48,90,95,95,96,77,89,97,95,90,6,11,15,14,8,71,22,68,87,88,13,74,93,95,88,85,26,87,73,28,59,90,85,92.0,45.0,59.0,92.0,84.0,158023,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,Neymar,25,Brazil,92,94,2100,94,56,96,82,95,92,75,81,96,89,84,9,9,15,15,11,62,36,61,75,77,21,81,90,88,81,80,33,90,78,24,53,80,83,88.0,46.0,59.0,88.0,79.0,190871,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,L. Suárez,30,Uruguay,92,92,2291,88,78,86,60,91,83,77,86,86,94,84,27,25,31,33,37,77,41,69,64,86,30,85,92,93,83,87,38,77,89,45,80,84,88,87.0,58.0,65.0,88.0,80.0,176580,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,M. Neuer,31,Germany,92,92,1493,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11,NaN,NaN,NaN,NaN,NaN,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Limiting dataset to one country

In [55]:
df_germany = df.loc[df['Nationality']=='Germany']

In [54]:
df.describe()

,Age,Overall,Potential,Special,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
count,17981.000000,17981.000000,17981.000000,17981.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,17981.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000,15952.000000
mean,25.144541,66.247984,71.190813,1594.095100,59.251755,55.550464,56.865283,59.030028,58.506833,207658.710138,59.251755,56.979689,55.550464,58.506833,56.865283,59.030028,60.057736,58.204050,59.359265,57.698721,59.251755,56.979689,55.550464,58.506833,56.865283,59.030028,60.057736,58.204050,59.359265,57.698721,58.204050
std,4.614272,6.987965,6.102199,272.151435,9.880164,12.192579,10.310178,9.926988,8.888040,32291.667313,9.880164,9.791627,12.192579,8.888040,10.310178,9.926988,9.349180,9.181392,9.978084,9.142825,9.880164,9.791627,12.192579,8.888040,10.310178,9.926988,9.349180,9.181392,9.978084,9.142825,9.181392
min,16.000000,46.000000,46.000000,728.000000,27.000000,25.000000,26.000000,27.000000,30.000000,16.000000,27.000000,30.000000,25.000000,30.000000,26.000000,27.000000,28.000000,31.000000,26.000000,31.000000,27.000000,30.000000,25.000000,30.000000,26.000000,27.000000,28.000000,31.000000,26.000000,31.000000,31.000000
25%,21.000000,62.000000,67.000000,1449.000000,53.000000,45.000000,49.000000,53.000000,53.000000,192622.000000,53.000000,50.000000,45.000000,53.000000,49.000000,53.000000,54.000000,52.000000,53.000000,51.000000,53.000000,50.000000,45.000000,53.000000,49.000000,53.000000,54.000000,52.000000,53.000000,51.000000,52.000000
50%,25.000000,66.000000,71.000000,1633.000000,60.000000,57.000000,58.000000,60.000000,59.000000,214057.000000,60.000000,58.000000,57.000000,59.000000,58.000000,60.000000,61.000000,59.000000,60.000000,58.000000,60.000000,58.000000,57.000000,59.000000,58.000000,60.000000,61.000000,59.000000,60.000000,58.000000,59.000000
75%,28.000000,71.000000,75.000000,1786.000000,66.000000,65.000000,65.000000,66.000000,65.000000,231448.000000,66.000000,64.000000,65.000000,65.000000,65.000000,66.000000,67.000000,65.000000,66.000000,64.000000,66.000000,64.000000,65.000000,65.000000,65.000000,66.000000,67.000000,65.000000,66.000000,64.000000,65.000000
max,47.000000,94.000000,94.000000,2291.000000,92.000000,87.000000,85.000000,92.000000,87.000000,241219.000000,92.000000,84.000000,87.000000,87.000000,85.000000,92.000000,90.000000,92.000000,91.000000,84.000000,92.000000,84.000000,87.000000,87.000000,85.000000,92.000000,90.000000,92.000000,91.000000,84.000000,92.000000


In [19]:
df_germany.describe()

,Age,Overall,Potential,Special,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
count,1140.000000,1140.000000,1140.000000,1140.000000,995.000000,995.000000,995.000000,995.000000,995.000000,1140.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000,995.000000
mean,24.202632,65.900877,71.579825,1564.554386,58.058291,55.833166,56.787940,57.784925,57.607035,213003.800877,58.058291,57.002010,55.833166,57.607035,56.787940,57.784925,59.055276,57.071357,58.212060,57.634171,58.058291,57.002010,55.833166,57.607035,56.787940,57.784925,59.055276,57.071357,58.212060,57.634171,57.071357
std,4.197097,6.877570,6.003492,277.128911,9.661818,12.275112,10.439162,9.732125,8.874678,28004.671297,9.661818,9.884646,12.275112,8.874678,10.439162,9.732125,9.276111,8.968979,9.830467,9.234963,9.661818,9.884646,12.275112,8.874678,10.439162,9.732125,9.276111,8.968979,9.830467,9.234963,8.968979
min,16.000000,46.000000,46.000000,736.000000,29.000000,27.000000,31.000000,30.000000,31.000000,2196.000000,29.000000,34.000000,27.000000,31.000000,31.000000,30.000000,30.000000,34.000000,28.000000,35.000000,29.000000,34.000000,27.000000,31.000000,31.000000,30.000000,30.000000,34.000000,28.000000,35.000000,34.000000
25%,21.000000,62.000000,67.000000,1428.750000,53.000000,45.000000,49.000000,52.000000,52.000000,199893.000000,53.000000,49.000000,45.000000,52.000000,49.000000,52.000000,54.000000,51.000000,52.500000,51.000000,53.000000,49.000000,45.000000,52.000000,49.000000,52.000000,54.000000,51.000000,52.500000,51.000000,51.000000
50%,24.000000,66.000000,71.000000,1598.000000,59.000000,58.000000,58.000000,59.000000,58.000000,215419.500000,59.000000,58.000000,58.000000,58.000000,58.000000,59.000000,60.000000,57.000000,59.000000,58.000000,59.000000,58.000000,58.000000,58.000000,58.000000,59.000000,60.000000,57.000000,59.000000,58.000000,57.000000
75%,27.000000,69.000000,75.000000,1749.250000,64.000000,66.000000,64.000000,64.000000,63.000000,236633.000000,64.000000,64.000000,66.000000,63.000000,64.000000,64.000000,65.000000,63.000000,65.000000,64.000000,64.000000,64.000000,66.000000,63.000000,64.000000,64.000000,65.000000,63.000000,65.000000,64.000000,63.000000
max,36.000000,92.000000,92.000000,2198.000000,86.000000,85.000000,83.000000,86.000000,87.000000,241085.000000,86.000000,81.000000,85.000000,87.000000,83.000000,86.000000,84.000000,83.000000,86.000000,81.000000,86.000000,81.000000,85.000000,87.000000,83.000000,86.000000,84.000000,83.000000,86.000000,81.000000,83.000000


In [56]:
print(df_germany.shape)
df_germany

(1140, 68)


,Name,Age,Nationality,Overall,Potential,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
4,M. Neuer,31,Germany,92,92,1493,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11,NaN,NaN,NaN,NaN,NaN,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,T. Kroos,27,Germany,90,90,2165,60,60,71,69,89,85,85,85,79,76,84,10,11,13,7,10,54,85,32,93,90,63,73,79,86,90,87,69,52,77,82,74,88,82,83.0,72.0,82.0,81.0,87.0,182521,83.0,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,CDM CM,83.0,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0
25,J. Boateng,28,Germany,88,88,1989,72,82,58,53,71,86,69,56,67,34,31,7,12,15,6,5,85,83,75,80,58,88,46,47,82,75,79,90,78,74,91,91,76,53,67.0,85.0,82.0,66.0,73.0,183907,67.0,81.0,85.0,73.0,82.0,66.0,69.0,65.0,65.0,79.0,CB,67.0,81.0,85.0,73.0,82.0,66.0,69.0,65.0,65.0,79.0,65.0
27,M. Hummels,28,Germany,88,88,2038,62,66,64,58,77,91,64,65,68,55,53,15,6,10,5,6,89,89,68,85,51,85,68,56,85,80,71,90,65,66,92,85,79,60,71.0,85.0,83.0,69.0,77.0,178603,71.0,80.0,85.0,77.0,83.0,69.0,70.0,69.0,68.0,78.0,CB,71.0,80.0,85.0,77.0,83.0,69.0,70.0,69.0,68.0,78.0,69.0
28,M. Özil,28,Germany,88,88,1927,75,48,83,70,90,78,82,84,86,74,79,6,14,10,6,14,54,24,52,84,76,22,67,83,87,89,70,19,68,69,16,57,92,77,85.0,41.0,57.0,82.0,79.0,176635,85.0,52.0,41.0,79.0,57.0,82.0,83.0,76.0,83.0,58.0,RW CAM,85.0,52.0,41.0,79.0,57.0,82.0,83.0,76.0,83.0,58.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17892,L. Tigges,18,Germany,49,64,736,18,20,24,20,13,29,13,10,11,7,10,49,51,48,47,50,13,9,36,21,6,7,11,6,39,15,19,13,26,25,11,51,30,7,NaN,NaN,NaN,NaN,NaN,238652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17900,T. Krahnert,20,Germany,49,63,1377,63,45,79,81,45,45,36,30,48,28,36,8,6,11,11,7,48,39,59,51,36,25,41,46,46,54,47,43,65,74,39,48,44,38,48.0,42.0,45.0,46.0,48.0,238954,48.0,46.0,42.0,48.0,45.0,46.0,49.0,45.0,47.0,47.0,CAM,48.0,46.0,42.0,48.0,45.0,46.0,49.0,45.0,47.0,47.0,45.0
17921,J. Schau,18,Germany,49,67,1381,61,54,63,74,43,48,40,39,40,29,33,11,11,8,13,14,44,40,63,46,32,37,40,38,52,55,46,51,60,76,45,55,41,27,45.0,47.0,48.0,44.0,47.0,239566,45.0,50.0,47.0,47.0,48.0,44.0,47.0,43.0,44.0,50.0,RM CDM,45.0,50.0,47.0,47.0,48.0,44.0,47.0,43.0,44.0,50.0,43.0
17930,C. Gnerlich,18,Germany,49,62,1397,65,51,82,89,46,41,39,39,48,25,35,15,6,12,10,8,45,41,67,46,25,43,36,30,54,53,48,48,63,62,48,46,43,29,46.0,47.0,48.0,44.0,46.0,240112,46.0,50.0,47.0,46.0,48.0,44.0,47.0,42.0,46.0,50.0,CDM,46.0,50.0,47.0,46.0,48.0,44.0,47.0,42.0,46.0,50.0,42.0


## Cleaning Data

In [58]:
target = "Overall"
predictors = list(df.columns)
for r in [target,"Name","Age","Nationality","ID","Preferred Positions"]:
  predictors.remove(r)
print(target,predictors)

Overall ['Potential', 'Special', 'Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing', 'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking', 'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties', 'Positioning', 'Reactions', 'Short passing', 'Shot power', 'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle', 'Strength', 'Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM', 'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM', 'RS', 'RW', 'RWB', 'ST']


In [86]:
df_germany = df_germany.fillna(0)

for c in predictors:
  df_germany[c] = df_germany[c].replace('[\\+*\-*(_)*,]', '', regex=True).astype(float)
  scaler = StandardScaler((0,1))
  df_germany[c] = scaler.fit_transform(np.array(df_germany[c]).reshape(-1,1))

## Naive Playing 11

In [87]:
df_germany[:11]

,Name,Age,Nationality,Overall,Potential,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
4,M. Neuer,31,Germany,92,3.402876,-0.258312,-0.067961,-0.154558,-0.285982,-0.730716,-0.156902,0.374602,-0.176341,-0.225722,-0.180075,-0.632661,-0.155421,1.919304,1.844779,3.219643,1.524089,1.720679,-0.203014,-0.649990,0.255865,-0.013745,-0.196439,-0.837487,-0.027206,-0.816811,0.287591,-0.138158,-0.769822,-0.389095,-0.063299,-0.116377,-0.369419,0.012004,0.365258,-0.226736,-2.374053,-2.230208,-2.328690,-2.368931,-2.404979,167495,-2.374053,-2.356062,-2.230208,-2.404979,-2.328690,-2.368931,-2.397535,-2.397340,-2.367768,-2.389668,GK,-2.374053,-2.356062,-2.230208,-2.404979,-2.328690,-2.368931,-2.397535,-2.397340,-2.367768,-2.389668,-2.397340
8,T. Kroos,27,Germany,90,3.069590,2.167616,-0.061329,-0.021893,0.120900,0.120450,0.241253,0.842445,0.096399,0.204878,0.089314,0.535974,0.119862,-0.246219,-0.215626,-0.176223,-0.277812,-0.234041,-0.017075,1.481848,-0.868544,0.142352,0.218639,0.408413,0.214810,0.631581,0.304412,1.034214,0.716128,0.167248,-0.108070,0.032240,0.225467,-0.021463,0.768438,0.208002,1.514482,1.064883,1.523886,1.435631,1.756395,182521,1.514482,1.243022,1.064883,1.756395,1.523886,1.435631,1.370136,1.308477,1.360421,1.315722,CDM CM,1.514482,1.243022,1.064883,1.756395,1.523886,1.435631,1.370136,1.308477,1.360421,1.315722,1.308477
25,J. Boateng,28,Germany,88,2.736305,1.532254,-0.021541,0.072256,-0.157493,-0.280099,0.066453,0.873634,0.034058,0.028999,0.023341,-0.243116,-0.080001,-0.326423,-0.189545,-0.093397,-0.299263,-0.357758,0.181687,1.404327,0.182534,0.082668,0.039146,0.996101,-0.036515,-0.060188,0.237129,0.531769,0.524392,0.368683,0.021268,0.018729,0.299828,0.041752,0.499651,0.030433,0.764885,1.659830,1.523886,0.731082,1.086749,183907,0.764885,1.479803,1.659830,1.086749,1.523886,0.731082,0.811963,0.730947,0.661386,1.363227,CB,0.764885,1.479803,1.659830,1.086749,1.523886,0.731082,0.811963,0.730947,0.661386,1.363227,0.730947
27,M. Hummels,28,Germany,88,2.736305,1.709145,-0.054698,0.003784,-0.029004,-0.154927,0.124720,1.029582,0.014577,0.083582,0.028839,0.146429,0.002961,-0.112544,-0.346032,-0.300462,-0.320714,-0.333014,0.207334,1.636891,0.011428,0.105623,-0.000118,0.925579,0.168269,0.134372,0.287591,0.699251,0.332657,0.368683,-0.043401,-0.017299,0.308090,0.019441,0.566848,0.073294,0.952284,1.659830,1.570868,0.871992,1.278077,178603,0.952284,1.432447,1.659830,1.278077,1.570868,0.871992,0.858477,0.923457,0.801193,1.315722,CB,0.952284,1.432447,1.659830,1.278077,1.570868,0.871992,0.858477,0.923457,0.801193,1.315722,0.923457
28,M. Özil,28,Germany,88,2.736305,1.308433,-0.011593,-0.073247,0.377878,0.145484,0.250964,0.624118,0.084710,0.198813,0.127798,0.498875,0.101007,-0.353158,-0.137383,-0.300462,-0.299263,-0.135068,-0.017075,-0.882554,-0.379670,0.101032,0.140111,-0.555397,0.158960,0.718052,0.321233,1.000718,0.308690,-0.312358,-0.028477,-0.003789,-0.319846,-0.084678,0.858033,0.177386,1.608182,-0.353837,0.349320,1.482601,1.373740,176635,1.608182,0.106469,-0.353837,1.373740,0.349320,1.482601,1.463165,1.260349,1.500228,0.365622,RW CAM,1.608182,0.106469,-0.353837,1.373740,0.349320,1.482601,1.463165,1.260349,1.500228,0.365622,1.260349
52,T. Müller,27,Germany,86,2.403019,1.853545,-0.008278,-0.034732,0.227974,0.195553,0.173275,0.780066,0.065228,0.180619,0.067323,0.740022,0.025587,-0.353158,-0.319951,-0.259049,-0.127653,-0.135068,0.162452,0.435310,0.329196,0.027575,0.145720,-0.343829,0.093802,0.934230,0.405337,0.732747,0.476458,-0.072555,0.021268,0.090786,-0.113288,-0.047493,0.611646,0.214125,1.420782,0.42

## ML Predicted Playing 11

In [88]:
X = df_germany[predictors]
y = df_germany[target]
print(X.shape,y.shape)

(1140, 62) (1140,)


In [89]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state= 1)
mlpReg = MLPRegressor(max_iter=1700)
mlpReg.fit(X_train.values,y_train.values)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1700,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [90]:
preds = mlpReg.predict(X_test)
score = r2_score(y_test,preds)
print(score)

-3.6640352620561227


In [91]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [92]:
preds_lr = lr.predict(X_test)
score_lr = r2_score(y_test,preds_lr)
print(score_lr) 

-72.815685223185
